In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/car-acceptability-classification-dataset/car.csv
/kaggle/input/car-acceptability-classification-dataset/car.data


In [2]:
cars=pd.read_csv("/kaggle/input/car-acceptability-classification-dataset/car.csv")

In [3]:
cars.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1728 entries, 0 to 1727
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   Buying_Price       1728 non-null   object
 1   Maintenance_Price  1728 non-null   object
 2   No_of_Doors        1728 non-null   object
 3   Person_Capacity    1728 non-null   object
 4   Size_of_Luggage    1728 non-null   object
 5   Safety             1728 non-null   object
 6   Car_Acceptability  1728 non-null   object
dtypes: object(7)
memory usage: 94.6+ KB


In [4]:
X=cars.drop("Car_Acceptability",axis=1)
y=cars.Car_Acceptability

In [5]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=100,random_state=42)

In [6]:
X_train.head()

,Buying_Price,Maintenance_Price,No_of_Doors,Person_Capacity,Size_of_Luggage,Safety
70,vhigh,vhigh,4,4,big,med
29,vhigh,vhigh,3,2,small,high
1540,low,med,3,2,small,med
69,vhigh,vhigh,4,4,big,low
1228,med,low,3,4,med,med


In [7]:
for key in cars.keys():
    print(key,np.unique(cars[key]))

Buying_Price ['high' 'low' 'med' 'vhigh']
Maintenance_Price ['high' 'low' 'med' 'vhigh']
No_of_Doors ['2' '3' '4' '5more']
Person_Capacity ['2' '4' 'more']
Size_of_Luggage ['big' 'med' 'small']
Safety ['high' 'low' 'med']
Car_Acceptability ['acc' 'good' 'unacc' 'vgood']


In [8]:
from catboost import CatBoostClassifier
clf = CatBoostClassifier()
clf.fit(X_train,y_train,cat_features=range(0,6),verbose=False)

In [9]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,clf.predict(X_test))

0.99